# Prompt Length and Complexity Management Tutorial

## Overview

This tutorial delves into strategies for optimizing prompt length and managing complexity when interacting with large language models (LLMs). We'll focus on two critical areas: crafting balanced prompts and tackling long-context challenges.

## Motivation

Mastering prompt engineering requires striking a delicate balance between providing sufficient context and maintaining conciseness. Additionally, many real-world scenarios involve processing extensive documents or intricate multi-step tasks that may exceed LLM context limits. Developing skills to address these challenges is essential for creating robust AI applications.

## Key Components

1. Optimizing prompt details and structure.
2. Techniques for handling extended contexts.
3. Practical demonstrations using Google's Gemini via OpenRouter and LangChain.

## Setup

First, let's import the necessary libraries and set up our environment.

In [5]:
from os import getenv

from dotenv import load_dotenv
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI

load_dotenv()

# Initialize the language model
llm = ChatOpenAI(
    openai_api_key=getenv("OPENROUTER_API_KEY"),
    openai_api_base=getenv("OPENROUTER_BASE_URL"),
    model_name="google/gemini-flash-1.5",
)

## Balancing Detail and Conciseness

Let's start by examining how to balance detail and conciseness in prompts. We'll compare responses from a detailed prompt and a concise prompt.

In [2]:
# Detailed prompt
detailed_prompt = PromptTemplate(
    input_variables=["topic"],
    template="""Please provide a comprehensive explanation of {topic}. Include its definition, 
    historical context, key components, practical applications, and any relevant examples. 
    Also, discuss any controversies or debates surrounding the topic, and mention potential 
    future developments or trends.""",
)

# Concise prompt
concise_prompt = PromptTemplate(
    input_variables=["topic"],
    template="Briefly explain {topic} and its main importance.",
)

topic = "artificial intelligence"

print("Detailed response:")
print(llm.invoke(detailed_prompt.format(topic=topic)).content)

print("\nConcise response:")
print(llm.invoke(concise_prompt.format(topic=topic)).content)

Detailed response:
## Artificial Intelligence: A Comprehensive Overview

Artificial intelligence (AI) is a broad field encompassing the theory and development of computer systems able to perform tasks that normally require human intelligence.  This includes tasks like learning, reasoning, problem-solving, perception, and natural language understanding.  Instead of explicitly programming every action, AI systems learn from data, adapt to new information, and improve their performance over time.

**Historical Context:**

The concept of AI dates back to antiquity, with myths and legends featuring artificial beings exhibiting intelligence. However, the formal field emerged in the mid-20th century.  Key milestones include:

* **1950s:** Alan Turing's seminal paper "Computing Machinery and Intelligence" proposed the Turing Test, a benchmark for machine intelligence.  The Dartmouth Workshop (1956) is widely considered the birth of AI as a field of study. Early successes focused on symbolic re

### Analysis of Prompt Balance

Let's analyze the differences between the detailed and concise prompts, and discuss strategies for finding the right balance.

In [3]:
analysis_prompt = PromptTemplate(
    input_variables=["detailed_response", "concise_response"],
    template="""Compare the following two responses on artificial intelligence:

Detailed response:
{detailed_response}

Concise response:
{concise_response}

Analyze the differences in terms of:
1. Information coverage
2. Clarity and focus
3. Potential use cases for each type of response

Then, suggest strategies for balancing detail and conciseness in prompts.""",
)

detailed_response = llm.invoke(detailed_prompt.format(topic=topic)).content
concise_response = llm.invoke(concise_prompt.format(topic=topic)).content

analysis = llm.invoke(
    analysis_prompt.format(
        detailed_response=detailed_response, concise_response=concise_response
    )
).content

print(analysis)

Here's an analysis comparing the two responses on artificial intelligence:

**1. Information Coverage:**

* **Detailed Response:** This response provides a much broader and deeper coverage of AI. It delves into the historical context, key components (ML, DL, NLP, Computer Vision, Robotics, Knowledge Representation), practical applications across multiple sectors, current controversies and ethical concerns, and future trends.  It offers specific examples of AI systems and provides a nuanced perspective on the challenges and opportunities.

* **Concise Response:** This response offers a very basic definition of AI and highlights its main benefits: automation, data analysis, and improved efficiency and decision-making.  It lacks the depth and breadth of the detailed response, omitting crucial aspects like the historical development, specific AI components, ethical concerns, and future trends.


**2. Clarity and Focus:**

* **Detailed Response:** While comprehensive, the detailed response 

## Strategies for Handling Long Contexts

Now, let's explore strategies for handling long contexts, which often exceed the token limits of language models.

### 1. Chunking

Chunking involves breaking long texts into smaller, manageable pieces. Let's demonstrate this using a long text passage.

In [4]:
# [A long passage about artificial intelligence, its history, applications, and future prospects...]

long_text = """
Artificial intelligence (AI) is a branch of computer science that aims to create intelligent machines that can simulate human cognitive processes.
The field of AI has a rich history dating back to the 1950s, with key milestones such as the development of the first neural networks and expert systems.
AI encompasses a wide range of subfields, including machine learning, natural language processing, computer vision, and robotics.
Practical applications of AI include speech recognition, image classification, autonomous vehicles, and medical diagnosis.
AI has the potential to revolutionize many industries, from healthcare and finance to transportation and entertainment.
However, there are ongoing debates and controversies surrounding AI, such as concerns about job displacement, bias in algorithms, and the ethical implications of autonomous systems.
Looking ahead, the future of AI holds promise for advancements in areas like explainable AI, AI ethics, and human-AI collaboration. 
The intersection of AI with other technologies like blockchain, quantum computing, and biotechnology will likely shape the future of the field.
But as AI continues to evolve, it is essential to consider the societal impact and ethical implications of these technologies.
One of the key challenges for AI researchers and developers is to strike a balance between innovation and responsibility, ensuring that AI benefits society as 
a whole while minimizing potential risks.
If managed effectively, AI has the potential to transform our world in ways we can only begin to imagine.
Though the future of AI is uncertain, one thing is clear: the impact of artificial intelligence will be profound and far-reaching.
"""

# Initialize the text splitter
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, length_function=len
)

# Split the text into chunks
chunks = text_splitter.split_text(long_text)

print(f"Number of chunks: {len(chunks)}")
print(f"First chunk: {chunks[0][:200]}...")

Number of chunks: 2
First chunk: Artificial intelligence (AI) is a branch of computer science that aims to create intelligent machines that can simulate human cognitive processes.
The field of AI has a rich history dating back to the...


### 2. Summarization

Summarization can be used to condense long texts while retaining key information. Let's use LangChain's summarization chain to demonstrate this.

In [6]:
# Convert text chunks to Document objects
doc_chunks = [Document(page_content=chunk) for chunk in chunks]

# Load the summarization chain
chain = load_summarize_chain(llm, chain_type="map_reduce")

# Summarize the long text
summary_result = chain.invoke(doc_chunks)

print("Summary:")
print(summary_result["output_text"])

/Users/tangquocthai/Works/personal/vn-ds/.venv/lib/python3.11/site-packages/langchain_openai/chat_models/base.py:360: UserWarning: Unexpected type for token usage: <class 'NoneType'>
  warnings.warn(f"Unexpected type for token usage: {type(new_usage)}")


Summary:
AI, rapidly evolving since the 1950s, offers transformative potential across industries but necessitates careful ethical consideration and responsible development to mitigate societal risks.  Interdisciplinary collaboration is key to harnessing its benefits.



### 3. Iterative Processing

For complex tasks that require multiple steps, we can use iterative processing. Let's demonstrate this with a multi-step analysis task.

In [8]:
def iterative_analysis(text, steps):
    """
    Perform iterative analysis on a given text.

    Args:
    text (str): The text to analyze.
    steps (list): List of analysis steps to perform.

    Returns:
    str: The final analysis result.
    """
    result = text
    for step in steps:
        prompt = PromptTemplate(
            input_variables=["text"],
            template=f"Analyze the following text. {step}\n\nText: {{text}}\n\nAnalysis:",
        )
        result = llm.invoke(prompt.format(text=result)).content
    return result


analysis_steps = [
    "Identify the main topics discussed.",
    "Summarize the key points for each topic.",
    "Provide a brief conclusion based on the analysis.",
]

final_analysis = iterative_analysis(long_text, analysis_steps)
print("Final Analysis:")
print(final_analysis)

Final Analysis:
Analysis: The provided text presents a balanced and comprehensive overview of artificial intelligence.  It effectively covers the key aspects of AI, from its historical context and technical underpinnings to its societal impact and ethical considerations. The structure is clear and logical, progressing from foundational concepts to future implications.  Each section offers a concise key takeaway, reinforcing the main points. The consistent emphasis on ethical considerations highlights a crucial and often overlooked aspect of AI development and deployment.

Conclusion: The text offers a strong, well-structured introduction to artificial intelligence, emphasizing the importance of responsible development and ethical considerations alongside its technological advancements and potential benefits.  It successfully conveys the complexity and multifaceted nature of AI, preparing the reader for a deeper exploration of the subject.

